In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape

In [2]:
# Load the data

data = pd.read_csv('ind1901_2015.csv')

In [3]:
# Extract the date rows and precipitation data
# Filter rows that contain date information, skipping 'x' and 'y' labels
date_rows = data[data['Unnamed: 0'].str.contains("X", na=False)]

In [4]:
# Reset index for date rows and drop the "Unnamed: 0" column for simplicity
date_rows = date_rows.reset_index(drop=True)
date_rows = date_rows.drop(columns=['Unnamed: 0'])

In [5]:
# Convert precipitation data to numeric, replacing any non-numeric entries with NaN, then fill NaNs with 0
date_rows = date_rows.apply(pd.to_numeric, errors='coerce').fillna(0)


In [6]:
# # Reshape data to a 3D format suitable for CNN input
# precipitation_data = date_rows.values.reshape(-1, date_rows.shape[1], 1)
precipitation_data = date_rows.values.flatten()

In [7]:
# Define a fixed threshold for extreme rainfall
threshold = 204.5
labels = (precipitation_data >= threshold).astype(int)  # Label as 1 if extreme, else 0

In [8]:
# Prepare features and labels
X = precipitation_data.reshape(-1,date_rows.shape[1],1, 1)  # Reshape to (samples, features)
y = labels.reshape(X.shape[0],-1)

In [9]:
X.shape
y.shape


(9125, 639)

In [10]:
# # # Preprocess the data
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)


In [19]:

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 1), activation='relu', input_shape=(date_rows.shape[1], 1, 1)),  # First convolutional layer
    MaxPooling2D((2, 2),padding='same'),  # Pooling layer
    Conv2D(64, (3, 1), activation='relu'),  # Second convolutional layer
    MaxPooling2D((2, 2),padding='same'),  # Pooling layer
    Flatten(),  # Flatten to convert 3D output to 1D
    Dense(128, activation='relu'),  # Fully connected layer
    Dropout(0.5),  # Dropout layer for regularization
    Dense(639)  # Output layer for regression
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [21]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.0069 - loss: 0.5156 - val_accuracy: 0.0000e+00 - val_loss: 0.0018
Epoch 2/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.0000e+00 - loss: 0.0018 - val_accuracy: 0.0000e+00 - val_loss: 0.0018
Epoch 3/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.0000e+00 - loss: 0.0017 - val_accuracy: 0.0000e+00 - val_loss: 0.0018
Epoch 4/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - accuracy: 0.0000e+00 - loss: 0.0017 - val_accuracy: 0.0000e+00 - val_loss: 0.0018
Epoch 5/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.0000e+00 - loss: 0.0017 - val_accuracy: 0.0000e+00 - val_loss: 0.0018
Epoch 6/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 0.0000e+00 - loss: 0.0014 - val_accuracy: 0.0000e+00 - val_loss: 0.0018
Epoch 7/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.0000e+00 - loss: 0.0015 - val_accuracy: 0.0000e+00 - val_loss: 0.0018
Epoch 8/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 8s 4

In [18]:
y_test.shape

(1825, 639)

In [22]:

# Predict on the test set
y_pred = model.predict(X_test)

# Reshape y_pred to match the number of outputs in y_test
y_pred = y_pred.reshape(None,y_test.shape[1])
# Reshape y_test to have a single column if it has multiple columns
# if y_test.ndim > 1 and y_test.shape[1] > 1:
#    y_test = y_test[:, -1]
  #  y_test = y_test.reshape(y_test.shape[0],)
  #  print("Warning: y_test was reshaped to a single column. Ensure this is intended.")


58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'R-squared: {r2:.2f}')

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'R-squared: {r2:.2f}')